### Submit an annotation job on Amazon SageMaker Ground Truth

**<font color=red>Important</font>: You should be running this notebook from a Sagemaker Notebook instance on AWS.** 

We have found that AWS will automatically insert quotation marks in some places, so you may need to remove those to be able to use this Jupyter Notebook through SageMaker

You can find detailed instructions for setting up AWS to run annotation jobs [here](docs/readme_groundTruthSetup.md). 

For this notebook, we will assume you've set up your Lambdas, uploaded your images and the annotation template to a bucket, and created a manifest file for that bucket.

In [ ]:
import MARS_AWS_helpers as mah
import sagemaker
import boto3
import time

In [ ]:
# name of the s3 buckets containing your data and the annotation instructions (.template file):
DATA_BUCKET = 'white-100'
TEMPLATE_BUCKET = 'my-annotation-instructions'

# create a data manifest of your bucket. This is a list of all the image files (.jpg only) that are in that bucket.
# If you've already done this, set MANIFEST to be the name of the manifest you created before--
# if you didn't change it, this should be "file_list.manifest".
MANIFEST = mah.create_manifest(DATA_BUCKET)

# path to the template for the annotation interface.
UITEMPLATE = f's3://{TEMPLATE_BUCKET}/annotation_interface.template'

First you'll need to retrieve Amazon Resource Names (ARNs). Think of these as codes that uniquely identify different resources on AWS. You will find these through the [AWS console](http://console.aws.amazon.com).

To find your `prehuman_arn`, search for and open the Lambdas service using the AWS search bar, select the pre-processing Lambda (usually has a name similar to serverlessrepo-aws-sagema-GtRecipePreHumanTaskFunc-XXXXXXXXXXXX), and copy the function ARN.

To find your `acs_arn`, search for and open the Lambdas service using the AWS search bar, select the post-processing Lambda (usually has a name similar to serverlessrepo-aws-sagema-GtRecipeAnnotationConsol-XXXXXXXXXXXXX), and copy the function ARN.

To find your `workteam_arn`, search for and open the SageMaker service using the AWS search bar, select <kbd>Labeling workforces</kbd> under Ground Truth on the left hand side menu, and copy ARN for Mechanical_turk_workforce_team. 
* If using a public workforce, find `Mechanical_turn_workforce_team` in the team summary, and copy the provided ARN.
* If using a private workforce, click the <kbd>Private</kbd> tab at the top of the page, then scroll down to the list of private teams, and copy the ARN of the team you wish to use. Please refer to [this tutorial](docs/readme_privateWorkforce.md) to learn how to create and add users to your own private team.

In [ ]:
prehuman_arn = 'arn:aws:lambda:us-east-2:522254924758:function:serverlessrepo-aws-sagema-GtRecipePreHumanTaskFunc-xxxxxxxxxxxxx'
acs_arn = 'arn:aws:lambda:us-east-1:919226420625:function:serverlessrepo-aws-sagema-GtRecipeAnnotationConsol-xxxxxxxxxxxxx'
workteam_arn = 'arn:aws:sagemaker:us-east-2:394669845002:workteam/public-crowd/default'

Next, we will package the task information. If you want to edit the number of annotations per image, the worker compensation, or the job description/keywords, do so here:

In [ ]:
# info about the data to be annotated and the annotation interface.
task = {'BUCKET': DATA_BUCKET,         
        'MANIFEST': MANIFEST, 
        'UITEMPLATE': UITEMPLATE, }

# additional task info/instructions for annotators.
task['info'] = {'task_description': 'Please label each body part of interest',
                'task_keywords': ['pose', 'keypoints','animal'],
                'task_title': 'Please label each body part of interest',
                'job_name': '{}-'.format(task['BUCKET']) + str(int(time.time())), 
                'time_limit': 45, } # time limit per frame in seconds

# Number of annotators per frame, and how much each annotator is paid per frame.
# Check AWS documentation for permissable compensation values. More workers will make your annotations
# more robust to outliers, while lower compensation often leads to noisier pose annotations.
task['price'] = {'num_workers': 5,
                 'dollars': 0,
                 'cents': 9,
                 'tenthcent': 0, }

# Insert ARNs for resources needed to run an image classification job.
task['arns'] = {'prehuman_arn': prehuman_arn,
                'acs_arn': acs_arn,
                'workteam_arn': workteam_arn, }

# public (Mechanical Turk), Vendor, or private job? Changes the fields sent to Amazon
task['workforce'] = 'public'
# task['workforce'] = 'vendor' # one of the vendors we have tried
# task['workforce'] = 'private' # private job -- tagged by folks in your organization

Now we are ready to submit the annotation job to SageMaker!

In [ ]:
# Connect to SageMaker
role = sagemaker.get_execution_role()

# Make sure the bucket is in the same region as this notebook.
mah.check_bucket_region(role, task)

# Package information/settings for the human workforce.
human_task_config = mah.configure_workforce(task)

# Format the task request.
ground_truth_request = mah.configure_ground_truth(task, human_task_config, role)

# Submit the task request.
sagemaker_client = boto3.client('sagemaker')
sagemaker_client.create_labeling_job(**ground_truth_request)

In the SageMaker console, click <kbd>Ground Truth</kbd>-><kbd>Labeling Jobs</kbd>; you should now see a job with name `task['info']['job_name']` in progress. You're done! Be sure to shut down this Notebook Instance so you don't continue to be billed for using it.